# Set up

In [ ]:
using IntervalArithmetic, Combinatorics, Polynomials, Serialization, LinearAlgebra, Plots, LaTeXStrings

In [ ]:
include("quadrature.jl")

We aim to find a self-similar solution to the viscous Burger's equation on $\mathbb{R}_+$

$$\partial_t v + v^2\partial_x v = \Delta v$$

The profile for 
$$\mathcal{L}u-\frac{u}{4}+u^2\frac{\partial u}{\partial x} = 0$$

The Vandermonde matrices below have been adapted to treat the above nonlinearity. The quadratures are still rigorous.

In [ ]:
n = 1500

# approximate solution
ū = interval.(deserialize("ubar"));

# regularised Vandermonde matrices for 2p and p+1 products
V̄4 = deserialize("V4r");
V̄6 = deserialize("V6r");
DV̄4 = deserialize("DV4r");
DV̄6 = deserialize("DV6r");

p = interval(3)
d = 1
α = d//2

# m = n+1, (n+1)ᵗʰ eigenvalue
λₘ = interval(d//2+n+1);
λ₀ = interval(d//2);

# generator
𝔏 = Diagonal(interval.(collect(0:n)).+λ₀);

In [ ]:
LinearAlgebra.norm(v::Vector) = sqrt(sum(v.^2))
function doublefac(n)
    if n==-1
        return 1
    else
        return doublefactorial(big(n))
    end
end

# computes Lᵖ/H¹ norms before taking fractional exponent
function L2(u)
    return sum(u.^2)
end

function H2(u)
    return sum((𝔏*u).^2)
end

function L6(u)
    return sum((V̄6*u).^4 .* (DV̄6*u).^2)
end

# rigorous upper bound of the 2-norm of a matrix
function op_norm(A)
    if size(A) == (2,2)
        Z = sqrt(sum(A.^2) + sqrt(((A[1,2]+A[2,1])^2+(A[1,1]-A[2,2])^2)*((A[1,2]-A[2,1])^2+(A[1,1]+A[2,2])^2)))/sqrt(interval(2))
        if isguaranteed(Z)
            return interval(sup(Z))
        else
            return Z
        end
    else
        all(isguaranteed.(A)) || error("matrix not guaranteed")
        return sqrt(interval(maximum(sup.(sum(abs.(A), dims = 1))))*interval(maximum(sup.(sum(abs.(A), dims = 2)))))
    end
end

# Proof

In [ ]:
# Gram matrix induced by ū∂ₓū
G = V̄4'*(((V̄4*ū).*(DV̄4*ū)).*V̄4);
# Non-Gram matrix induced by ū²∂ₓ
DG = V̄4'*(((V̄4*ū).^2).*DV̄4);

In [ ]:
# PₙF(ū)
PFū = ū + 𝔏\(-ū/interval(4)+V̄4'*((V̄4*ū).^2 .*(DV̄4*ū)));
println(norm(PFū))

In [ ]:
# PₙDF(ū)Pₙ
DFū = interval.(I(n+1)) - inv(𝔏)/interval(4) + 𝔏\(interval(2)*G+DG);
# approximate numerical inverse of PₙDF(ū)Pₙ
Aₙ = interval.(inv(mid.(DFū)));

In [ ]:
# rigorous L²-norms of ūψ̂ⱼ∂ₓū
int = sum((((V̄6*ū).*(DV̄6*ū)).*V̄6).^2, dims = 1);
# rigorous L²-norms of ū²∂ₓψ̂ⱼ
Dint = sum((((V̄6*ū).^2).*DV̄6).^2, dims = 1);

In [ ]:
w = interval(2)*sqrt.([int[i] - L2(G[:,i]) for i=1:n+1])+sqrt.([Dint[i] - L2(DG[:,i]) for i=1:n+1]);

In [ ]:
Y = sqrt(H2(Aₙ*PFū)+(L6(ū)-L2(V̄4'*((V̄4*ū).^2 .*(DV̄4*ū)))))

In [ ]:
Z¹¹ = interval(op_norm(𝔏*(interval.(I(n+1)) - Aₙ*DFū)*inv(𝔏)));

In [ ]:
Z²¹ = norm(𝔏\w);

In [ ]:
ψ̂₀ = interval.(Float64, sqrt.([interval(BigFloat,doublefac(2*m-1)//2^m//factorial(m)) for m=0:big(n)]/sqrt(interval(BigFloat,π))));
# sup∂ₓψ̂ = (supψ̂ + 2*sqrt.(1 .- 0.5 ./interval.(1:n+1)).*supψ̂)./sqrt(interval(2)*exp(interval(1)));

In [ ]:
supψ̂ = [interval(BigFloat, (2^(2*k+1)- binomial(2*k, k))*factorial(k)//2^k)/sqrt(interval(BigFloat, factorial(2*k))) for k=0:big(n)]/interval(BigFloat, π)^interval(1//4);

In [ ]:
sup∂ₓψ̂ = [sqrt(interval((2*k+1)*factorial(2*k+1)//4^k))/interval(factorial(k))*exp(interval(big(-0.5))) for k=0:big(n)]/interval(BigFloat, π)^interval(1//4);

In [ ]:
supū = interval(Float64, sum(abs.(ū).*supψ̂))

In [ ]:
sup∂ₓū = interval(Float64, sum(abs.(ū).*sup∂ₓψ̂))

In [ ]:
Sₙ = (interval(pi)*log(interval(4)) - sum([interval(BigFloat, doublefac(2*m-1)//(m+1//2)^2//factorial(m)//2^m) for m=0:big(n)]))/
sqrt(interval(π))

In [ ]:
ū₀ = sum(ū.*ψ̂₀)

In [ ]:
Dint = sum((((V̄6*ū).^2).*(DV̄6+V̄6)).^2, dims = 1)

In [ ]:
DG = V̄4'*(((V̄4*ū).^2).*(V̄4+DV̄4));

In [ ]:
w̃ = ū₀^2*sqrt(Sₙ)*ψ̂₀+sqrt.(abs.([Dint[i] - L2(DG[:,i]) for i=1:n+1]))/λₘ;

In [ ]:
Z¹² = norm(abs.(𝔏*interval.(Aₙ)*inv(𝔏))*w̃);

In [ ]:
Z²² = (interval(1//4)+interval(2)*supū*sup∂ₓū)/λₘ + supū^2/sqrt(λₘ);

In [ ]:
Z₁ = op_norm([Z¹¹ Z¹² ; Z²¹ Z²²])

In [ ]:
if sup(Z₁)>=1
    println("ū cannot be validated")
end

In [ ]:
[Z¹¹ Z¹² ; Z²¹ Z²²]

In [ ]:
op_n = interval(max(op_norm(𝔏*Aₙ*inv(𝔏)), interval(1)))

In [ ]:
Z₂ = interval(2)^interval(15//4)*op_n*(sqrt(interval(2))*supū+sup∂ₓū)

In [ ]:
Z₃ = interval(96)*op_n

In [ ]:
δ̄ = (-Z₂+sqrt(Z₂^2 +interval(2)*Z₃-interval(2)*Z₁*Z₃))/Z₃

In [ ]:
P(δ) = Z₃/interval(6)*δ^3 + Z₂/interval(2)*δ^2 - (interval(1) - Z₁)*δ + Y

In [ ]:
δ̲ = Y/(interval(1)-Z₁)*(interval(1.17163))
if sup(P(δ̲))<0
    println("δ̲ is validated")
end

# Check positivity (under regularity assumption)

In [ ]:
# infinity nor error
inf_err = interval(2)^interval(7//4)*δ̲

In [ ]:
m = 10
supm = inf_err+sum(abs.((supψ̂.*ū)[m+2:end]))
Pₘ = zeros(Interval{BigFloat},m+1)
for j = 0:m
#     println(j)
    Lⱼ =  [interval((-1)^i)*interval(BigFloat, binomial(j,i)//factorial(i)) for i=0:j]
    Pₘ[1:j+1] += ū[Int64(j+1)]*Lⱼ
end

In [ ]:
r₀ = interval(BigFloat,4.0)
z = mince(interval(0,r₀.^2/interval(4)), 2000)

if all(inf.(evalpoly.(z,tuple(Pₘ)).*exp.(-z).-supm.*exp.(-z/interval(2))).>0) && inf(r₀^2/interval(16)-interval(4)*r₀*((supū+inf_err)*exp(-r₀^2/interval(8)))^2)>0
    println("positivity checked")
else
    println("positivity not checked")
end

# Plotting

In [ ]:
ū = interval.(deserialize("ubar"));

In [ ]:
setprecision(1024)
n =1000
ū = mid.(deserialize("ubar"))
Zs = sqrt.(sqrt(big(π))).*[big(doublefac(2*j-1)//2^(j)//factorial(j)) for j=0:big(n)];
facsp1 = [doublefac(2*k-1)*factorial(k) for k=0:big(n)];

In [ ]:
#solution plotting

rloc = collect(0:400)/100
zloc = rloc.^2/4
sol = zeros(length(zloc), Threads.nthreads());
for j in 0:big(n)
    dfj = doublefac(2*j-1)
    Lⱼ =  Polynomial([(-1)^k*big(dfj//facsp1[k+1]//factorial(j-k)//2^(j-k)) for k=0:j]);
    sol[:,Threads.threadid()] .+= (mid(ū[j+1]) .* Lⱼ.(zloc))/Zs[j+1]
end

In [ ]:
plot(rloc, sum(sol, dims = 2).*exp.(-zloc), xlabel = L"r", ylabel = L"\bar{u}", legend = false, dpi =1000)

In [ ]:
png("Burger_sol")